In [1]:
library(mice)
library(finalfit)
library(ggplot2)
library(dplyr)
library(stats)
library(miceadds)
library(missMethods)
library(smotefamily)
library(caret)
library(cowplot)
library(mlr)
library(fastDummies)
library(DMwR2)
library(mlmi)
library(ranger)
library(scales)
library(Hmisc)

Warning message:
"pakiet 'mice' został zbudowany w wersji R 4.3.2"

Dołączanie pakietu: 'mice'


Następujący obiekt został zakryty z 'package:stats':

    filter


Następujące obiekty zostały zakryte z 'package:base':

    cbind, rbind


Warning message:
"pakiet 'ggplot2' został zbudowany w wersji R 4.3.3"

Dołączanie pakietu: 'dplyr'


Następujące obiekty zostały zakryte z 'package:stats':

    filter, lag


Następujące obiekty zostały zakryte z 'package:base':

    intersect, setdiff, setequal, union


* miceadds 3.16-18 (2023-01-06 10:54:00)

Ładowanie wymaganego pakietu: lattice

Warning message:
"pakiet 'cowplot' został zbudowany w wersji R 4.3.2"
Warning message:
"pakiet 'mlr' został zbudowany w wersji R 4.3.2"
Ładowanie wymaganego pakietu: ParamHelpers

Warning message:
"pakiet 'ParamHelpers' został zbudowany w wersji R 4.3.2"
Warning message: 'mlr' is in 'maintenance-only' mode since July 2019.
Future development will only happen in 'mlr3'
(<https://mlr3.mlr-org.com>). Due to t

In [24]:
create_zero_df <- function(num_rows, num_cols, colnames) {
  #' Create dataframe of zeros.
  
  df <- as.data.frame(matrix(0, nrow = num_rows, ncol = num_cols))
  names(df) <- colnames
  return(df)
}

In [25]:
test_impute_mice <- function(data, m, maxit, method, formula, family, intcols) {
    #' Compute statistical estimates for data imputed with mice package.

    if (m > 1) {
        imp <- mice(data, method=method, m=m, maxit=maxit, printFlag=FALSE)
        imp[intcols] <- imp[intcols] %>% lapply(as.integer)
        fit <- with(imp, glm(formula=as.formula(formula), family=family))
        tab <- summary(pool(fit), "all", conf.int = TRUE)
        tab <- tab[, c("term", "estimate", "2.5 %", "97.5 %")]
        tab <- as.data.frame(tab)
    }

    else {
        imp <- mice(data, method=method, m=m, maxit=maxit, printFlag=FALSE) %>% complete()
        imp[intcols] <- imp[intcols] %>% lapply(as.integer)
        fit <- glm(formula=formula, family=family, data=imp)

        coefs <- as.data.frame(fit$coefficients)
        colnames(coefs) <- c("estimate")
        coefs <- cbind(term=rownames(coefs), coefs)
        rownames(coefs) <- NULL

        conf <- confint(fit)
        rownames(conf) <- NULL
        
        tab <- cbind(coefs, conf)
    }
    
    return(tab)
}

In [26]:
simulate_mice <- function(nsim, data, m, maxit, method, formula, family, intcols, nvars) {
    #' Simulate statistical estimates computation for data imputed with mice package.

    nrows <- nvars + 1
    
    tst_imp <- create_zero_df(nrows*nsim, 4, c("term", "estimate", "2.5 %", "97.5 %"))

    for (n in 1:nsim) {
        data_n <- sample_n(data, size=nrow(data), replace=TRUE)
        start_index <- (n - 1) * nrows + 1
        end_index <- n * nrows
        tst_imp_n <- test_impute_mice(data_n, m, maxit, method, formula, family, intcols)
        tst_imp[start_index:end_index, ] <- tst_imp_n
    }

    return(tst_imp)
}

In [27]:
pool_mlmi_helper <- function(data, formula, family) {
    #' Helper function for pooling estimates after MLMI imputation.

    fit <- glm(formula=as.formula(formula), family=family, data=data)
    coefs <- as.data.frame(fit$coefficients)
    colnames(coefs) <- c("estimate")
    coefs <- cbind(term=rownames(coefs), coefs)
    rownames(coefs) <- NULL

    conf <- confint(fit)
    rownames(conf) <- NULL
    
    res <- cbind(coefs, conf)

    return(res)
}

In [28]:
pool_mlmi <- function(imp, M, formula, family) {
    #' Pool statistical estimates for data imputed with MLMI.

    data <- imp[[1]]
    res <- pool_mlmi_helper(data, formula, family)

    for (i in 2:M) {
        data_i <- imp[[i]]
        res_i <- pool_mlmi_helper(data_i, formula, family)
        res <- rbind(res, res_i)
    }

    pooled <- aggregate(res[, c(2, 3, 4)], list(res$term), FUN=mean)
    colnames(pooled)[1] <- "term"

    return(pooled)

}

In [29]:
test_impute_mlmi <- function(data, M, nCat, rseed, formula, family, bcols, fcols, intcols) {
    #' Compute statistical estimates for data imputed with MLMI.
    
    imp <- mixImp(obsData = data, nCat = nCat, M = M, rseed = rseed)
    
    for (i in 1:M) {

        for (bcol in bcols) {
            imp[[i]][bcol] <- ifelse(imp[[i]][bcol] == 1, 0, 1)
        }

        imp[[i]][fcols] <- imp[[i]][fcols] %>% lapply(factor)
        imp[[i]][intcols] <- imp[[i]][intcols] %>% lapply(as.integer)
    }
    
    pooled <- pool_mlmi(imp, M, formula, family)

    return(pooled)
}

In [30]:
simulate_mlmi <- function(nsim, data, M, nCat, formula, family, bcols, fcols, intcols, nvars) {
    #' Simulate statistical estimates computation for data imputed with MLMI.
    
    nrows <- nvars + 1
    
    tst_imp <- create_zero_df(nrows*nsim, 4, c("term", "estimate", "2.5 %", "97.5 %"))

    for (n in 1:nsim) {
        data_n <- sample_n(data, size=nrow(data), replace=TRUE)
        rseed <- sample(1:5000, 1)
        tst_imp_n <- test_impute_mlmi(data_n, M, nCat, rseed, formula, family, bcols, fcols, intcols)
        start_index <- (n - 1) * nrows + 1
        end_index <- n * nrows
        tst_imp[start_index:end_index, ] <- tst_imp_n
    }

    return(as.data.frame(tst_imp))
}

In [31]:
test_impute_knn <- function(data, formula, family, intcols) {
    #' Compute statistical estimates for data imputed with KNN.

    imp <- knnImputation(data, scale=FALSE)
    imp[intcols] <- imp[intcols] %>% lapply(as.integer)

    fit <- glm(formula=formula, family=family, data=imp)

    coefs <- as.data.frame(fit$coefficients)
    colnames(coefs) <- c("estimate")
    coefs <- cbind(term=rownames(coefs), coefs)
    rownames(coefs) <- NULL

    conf <- confint(fit)
    rownames(conf) <- NULL
    
    tab <- cbind(coefs, conf)

    return(tab)
}

In [32]:
simulate_knn <- function(nsim, data, formula, family, intcols, nvars) {
    #' Simulate statistical estimates computation for data imputed with KNN.

    nrows <- nvars + 1
    
    tst_imp <- create_zero_df(nrows*nsim, 4, c("term", "estimate", "2.5 %", "97.5 %"))

    for (n in 1:nsim) {
        data_n <- sample_n(data, size=nrow(data), replace=TRUE)
        tst_imp_n <- test_impute_knn(data_n, formula, family, intcols)
        start_index <- (n - 1) * nrows + 1
        end_index <- n * nrows
        tst_imp[start_index:end_index, ] <- tst_imp_n
    }

    return(as.data.frame(tst_imp))
    
}

In [33]:
PB <- function(sim_imp, real_coefs) {
    #' Compute percent bias.

    exp_est <- aggregate(sim_imp$estimate, list(sim_imp$term), FUN=mean)
    rownames(exp_est) <- exp_est$Group.1
    exp_est <- subset(exp_est, select=-1)

    terms <- unique(sim_imp$term)
    pbs <- rep(0, length(terms))

    for (t in 1:length(terms)) {
        pbs[t] <- 100*abs((exp_est[terms[t],] - as.numeric(real_coefs[terms[t]])) / as.numeric(real_coefs[terms[t]]))
    }
    
    return(pbs)
}

In [34]:
CR <- function(sim_imp, real_coefs) {
    #' Compute coverage rate.

    terms <- unique(sim_imp$term)
    crs <- rep(0, length(terms))

    for (t in 1:length(terms)) {
        tsubs <- subset(sim_imp, term==terms[t])
        crs[t] <- mean(tsubs["2.5 %"] < as.numeric(real_coefs[terms[t]]) & as.numeric(real_coefs[terms[t]]) < tsubs["97.5 %"])
    }

    return(crs)
}

In [35]:
AW <- function(sim_imp, real_coefs) {
    #' Compute average width of confidence intervals.

    terms <- unique(sim_imp$term)
    aws <- rep(0, length(terms))

    for (t in 1:length(terms)) {
        tsubs <- subset(sim_imp, term==terms[t])
        diff <- tsubs["97.5 %"] - tsubs["2.5 %"]
        aws[t] <- mean(diff[, 1])
    }

    return(aws)   
}

In [36]:
fram <- read.csv("Framingham.csv")
fram_mcar <- read.csv("fram_mcar_cat.csv")

In [37]:
fcols <- c("male", "education", "currentSmoker", "BPMeds", "prevalentStroke", "prevalentHyp", "diabetes", "TenYearCHD")
bcols <- c("male", "currentSmoker", "BPMeds", "prevalentStroke", "prevalentHyp", "diabetes", "TenYearCHD")
intcols <- c("age", "cigsPerDay", "glucose", "totChol", "heartRate")
fram_mcar[fcols] <- fram_mcar[fcols] %>% lapply(factor)
fram[fcols] <- fram[fcols] %>% lapply(factor)

In [38]:
glm_fit <- glm(formula = TenYearCHD ~ diabetes, family = "binomial", data = fram)

In [39]:
real_coefs <- glm_fit$coefficients
real_coefs

(Intercept)   diabetes1 
  -1.763120    1.398477

##### Logistic regression imputation

In [40]:
method1 <- rep("", ncol(fram_mcar))
method1[9] <- "logreg"

In [79]:
sim_imp1 <- simulate_mice(1000, fram_mcar, 1, 1, method1, "TenYearCHD ~ diabetes", "binomial", intcols, 1)

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowa

In [80]:
pb1 <- PB(sim_imp1, real_coefs)
pb1

[1] 0.07336312 9.32173248

In [81]:
cr1 <- CR(sim_imp1, real_coefs)
cr1

[1] 0.945 0.868

In [82]:
aw1 <- AW(sim_imp1, real_coefs)
aw1

[1] 0.2224477 1.0116729

##### KNN imputation

In [83]:
sim_imp8 <- simulate_knn(1000, fram_mcar, "TenYearCHD ~ diabetes", "binomial", intcols, 1)

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowa

In [84]:
pb8 <- PB(sim_imp8, real_coefs)
pb8

[1] 0.3852412 8.9406372

In [85]:
cr8 <- CR(sim_imp8, real_coefs)
cr8

[1] 0.946 0.911

In [86]:
aw8 <- AW(sim_imp8, real_coefs)
aw8

[1] 0.2213835 1.0838910

##### MLMI

In [87]:
catfram <- fram_mcar[, (names(fram_mcar) %in% fcols)]
contfram <- fram_mcar[, !(names(fram_mcar) %in% colnames(catfram))]
fram_reorder <- cbind(catfram, contfram)
fram_reorder[] <- lapply(fram_reorder, function(x) if(is.factor(x)) as.integer(x) else x)

In [88]:
sim_imp9 <- simulate_mlmi(1000, fram_reorder, 10, 8, "TenYearCHD ~ diabetes", "binomial", bcols, fcols, intcols, 1)

Imputing categorical variables using all 2-way associations.

Imputing continuous variables assuming main effects of categorical variables



Steps of ECM: 
1...2...3...4...5...6...7...8...9...10...11...12...


Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Imputing categorical variables using all 2-way associations.

Imputing continuous variables assuming main effects of categorical variables



Steps of ECM: 
1...2...3...4...5...6...7...8...9...10...11...12...13...14...15...


Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Imputing categorical variables using all 2-way associations.

Imputing continuous variables assuming main effects of categorical variables



...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Imputing categorical variables using all 2-way associations.

Imputing continuous variables assuming main effects of categorical variables



Steps of ECM: 
1...2...3...4...5...6...7...8...9...10...11...12...13...14...15...16...


Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Imputing categorical variables using all 2-way associations.

Imputing continuous variables assuming main effects of categorical variables



Steps of ECM: 
1...2...3...4...5...6...7...8...9...10...11...12...13...14...15...16...17...18...19...20...21...22...23...


Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...



In [89]:
pb9 <- PB(sim_imp9, real_coefs)
pb9

[1] 0.472230 8.603574

In [90]:
cr9 <- CR(sim_imp9, real_coefs)
cr9

[1] 0.947 0.920

In [91]:
aw9 <- AW(sim_imp9, real_coefs)
aw9

[1] 0.2226706 1.0416684

##### Decision tree imputation

In [92]:
method10 <- rep("", ncol(fram_mcar))
method10[9] <- "cart"

In [93]:
sim_imp10 <- simulate_mice(1000, fram_mcar, 1, 1, method10, "TenYearCHD ~ diabetes", "binomial", intcols, 1)

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowa

In [94]:
pb10 <- PB(sim_imp10, real_coefs)
pb10

[1] 0.2978077 4.0919405

In [95]:
cr10 <- CR(sim_imp10, real_coefs)
cr10

[1] 0.937 0.938

In [96]:
aw10 <- AW(sim_imp10, real_coefs)
aw10

[1] 0.2225749 1.0330628

##### Random forest imputation

In [97]:
method11 <- rep("", ncol(fram_mcar))
method11[9] <- "rf"

In [98]:
sim_imp11 <- simulate_mice(1000, fram_mcar, 1, 1, method11, "TenYearCHD ~ diabetes", "binomial", intcols, 1)

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowa

In [99]:
pb11 <- PB(sim_imp11, real_coefs)
pb11

[1]  0.611354 10.313331

In [100]:
cr11 <- CR(sim_imp11, real_coefs)
cr11

[1] 0.939 0.912

In [101]:
aw11 <- AW(sim_imp11, real_coefs)
aw11

[1] 0.2207319 1.1641318

##### Summary

In [105]:
pbs <- rbind(c("logistic regression", pb1), c("KNN", pb8), c("MLMI", pb9), c("decision tree", pb10), c("random forest", pb11))
pbs <- as.data.frame(pbs)
colnames(pbs)[2:3] <- names(real_coefs)
pbs[,2:3] <- pbs[,2:3] %>% lapply(as.numeric)
pbs <- subset(pbs, select=-c(2))
pbs

,V1,diabetes1
,<chr>,<dbl>
1,logistic regression,9.321732
2,KNN,8.940637
3,MLMI,8.603574
4,decision tree,4.091940
5,random forest,10.313331


In [106]:
crs <- rbind(c("mean", cr1), c("KNN", cr8), c("MLMI", cr9), c("decision tree", cr10), c("random forest", cr11))
crs <- as.data.frame(crs)
colnames(crs)[2:3] <- names(real_coefs)
crs[,2:3] <- crs[,2:3] %>% lapply(as.numeric)
crs <- subset(crs, select=-c(2))
crs

,V1,diabetes1
,<chr>,<dbl>
1,mean,0.868
2,KNN,0.911
3,MLMI,0.920
4,decision tree,0.938
5,random forest,0.912


In [107]:
aws <- rbind(c("mean", aw1), c("KNN", aw8), c("MLMI", aw9), c("decision tree", aw10), c("random forest", aw11))
aws <- as.data.frame(aws)
colnames(aws)[2:3] <- names(real_coefs)
aws[,2:3] <- aws[,2:3] %>% lapply(as.numeric)
aws <- subset(aws, select=-c(2))
aws

,V1,diabetes1
,<chr>,<dbl>
1,mean,1.011673
2,KNN,1.083891
3,MLMI,1.041668
4,decision tree,1.033063
5,random forest,1.164132


In [ ]:
+